In [ ]:
!pip install pulp

In [ ]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, lpSum, LpBinary, LpMaximize, PULP_CBC_CMD, LpStatus
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

pd.set_option("display.max_columns", None)
sns.set(style="whitegrid")

In [ ]:
df_base = pd.read_csv("optimized_allocation_results.csv")

colunas_esperadas = ["id", "predicted_proba", "risk_category", "simulated_cost", "true_label"]
colunas_presentes = all(col in df_base.columns for col in colunas_esperadas)

print("Columns found:", list(df_base.columns))
print("Valid structure for simulation?" , colunas_presentes)

df_base.head(10)

In [ ]:
def simular_alocacao(df, capacidade_max, orcamento_total, min_alto_risco,
                     peso_risco=1.0, peso_custo=0.001, verbose=False):
    """
    Runs an optimal allocation simulation based on the given parameters.
    Return: DataFrame with decisions + solution metrics.
    """
    df_sim = df.copy()
    ids = df_sim["id"].tolist()
    risco = dict(zip(df_sim["id"], df_sim["predicted_proba"]))
    custo = dict(zip(df_sim["id"], df_sim["simulated_cost"]))
    categoria = dict(zip(df_sim["id"], df_sim["risk_category"]))

    modelo = LpProblem("Simulacao_Alocacao", LpMaximize)
    x = {i: LpVariable(f"x_{i}", cat=LpBinary) for i in ids}

    modelo += lpSum([peso_risco * risco[i] * x[i] - peso_custo * custo[i] * x[i] for i in ids])

    modelo += lpSum([x[i] for i in ids]) <= capacidade_max, "Restricao_Capacidade"
    modelo += lpSum([custo[i] * x[i] for i in ids]) <= orcamento_total, "Restricao_Orcamento"
    ids_alto = [i for i in ids if categoria[i] == "alto"]
    modelo += lpSum([x[i] for i in ids_alto]) >= min_alto_risco, "Restricao_Alto_Risco"

    status = modelo.solve(PULP_CBC_CMD(msg=False))
    df_sim["selecionado"] = df_sim["id"].apply(lambda i: int(x[i].varValue))

    atendidos = df_sim["selecionado"].sum()
    risco_medio = df_sim[df_sim["selecionado"] == 1]["predicted_proba"].mean()
    custo_total = df_sim[df_sim["selecionado"] == 1]["simulated_cost"].sum()
    positivos = df_sim[(df_sim["selecionado"] == 1) & (df_sim["true_label"] == 1)].shape[0]
    altos = df_sim[(df_sim["selecionado"] == 1) & (df_sim["risk_category"] == "alto")].shape[0]

    if verbose:
        print(f"Status: {LpStatus[modelo.status]}")
        print(f"Served: {atendidos}")
        print(f"Medium risk: {risco_medio:.4f}")
        print(f"Total cost: R$ {custo_total}")
        print(f"Positives included: {positivos}")
        print(f"High risk included: {altos}")

    return df_sim, {
        "status": LpStatus[modelo.status],
        "atendidos": atendidos,
        "risco_medio": risco_medio,
        "custo_total": custo_total,
        "positivos": positivos,
        "alto_risco": altos
    }


In [ ]:
cenario_teste = {
    "capacidade_max": 30,
    "orcamento_total": 18000,
    "min_alto_risco": 10,
    "peso_risco": 1.0,
    "peso_custo": 0.001
}

df_simulado, metricas = simular_alocacao(df_base, **cenario_teste, verbose=True)
df_simulado[df_simulado["selecionado"] == 1].head(10)

In [ ]:
capacidades = [20, 30, 40, 50]
orcamentos = [10000, 15000, 18000, 20000, 25000]
altos_risco = [0, 5, 10]
pesos_custo = [0.0, 0.001]

cenarios = []
id_cenario = 1

for cap, orc, risco_min, p_custo in product(capacidades, orcamentos, altos_risco, pesos_custo):
    cenarios.append({
        "nome": f"Cen_{id_cenario:03d}",
        "capacidade_max": cap,
        "orcamento_total": orc,
        "min_alto_risco": risco_min,
        "peso_risco": 1.0,
        "peso_custo": p_custo
    })
    id_cenario += 1

print(f"Total number of scenarios generated: {len(cenarios)}")

In [ ]:
resultados = []

for c in cenarios:
    _, met = simular_alocacao(df_base,
                               capacidade_max=c["capacidade_max"],
                               orcamento_total=c["orcamento_total"],
                               min_alto_risco=c["min_alto_risco"],
                               peso_risco=c["peso_risco"],
                               peso_custo=c["peso_custo"])
    met["cenario"] = c["nome"]
    met["capacidade_max"] = c["capacidade_max"]
    met["orcamento_total"] = c["orcamento_total"]
    met["min_alto_risco"] = c["min_alto_risco"]
    met["peso_custo"] = c["peso_custo"]
    resultados.append(met)

df_resultados = pd.DataFrame(resultados)
df_resultados.head(10)

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_resultados, x="custo_total", y="risco_medio", hue="peso_custo", palette="Set2")
plt.title("Average Risk vs Total Cost by Scenario")
plt.xlabel("Total Allocation Cost (R$)")
plt.ylabel("Average Risk of Selected Candidates")
plt.legend(title="Weight Cost")
plt.tight_layout()
plt.show()

In [ ]:
df_resultados['custo_total'].plot(kind='hist', bins=20, title='custo_total')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
df_resultados['custo_total'].plot(kind='line', figsize=(8, 4), title='custo_total')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
df_resultados['atendidos'].plot(kind='line', figsize=(8, 4), title='atendidos')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
df_resultados.to_csv("simulation_results_120cenarios.csv", index=False)

In [ ]:
df_resultados.info()

In [ ]:
df_resultados.describe()